In [1]:
import numpy as np
import tensorflow as tf
import tfnumpy as tfnp

In [15]:
from itertools import product
import random

In [2]:
x1 = np.array([[1, 0], [0, 1], [2, -1.4]], dtype=np.float32)
x2 = np.array([[0, -1], [1, 0], [-1.5, 0.6]], dtype=np.float32)
x = np.zeros((3, 2, 2))
x[:, :, 0] = x1
x[:, :, 1] = x2

# Using numpy first

In [3]:
a = np.random.uniform(size=x.shape[2])
b = np.random.uniform(size=x.shape[2])

In [4]:
x[:, :, 0]*a[0]+x[:, :, 1]*a[1]

array([[ 0.12972818, -0.43167692],
       [ 0.43167692,  0.12972818],
       [-0.38805902,  0.07738671]])

In [5]:
Ta = sum(x[:, :, i]*a[i] for i in range(x.shape[2]))
Ta

array([[ 0.12972818, -0.43167692],
       [ 0.43167692,  0.12972818],
       [-0.38805902,  0.07738671]])

In [6]:
Tb = sum(x[:, :, i]*b[i] for i in range(x.shape[2]))
Tb

array([[ 0.02117039, -0.48500233],
       [ 0.48500233,  0.02117039],
       [-0.68516272,  0.26136287]])

In [7]:
eigvals_a, eigvecs_a = np.linalg.eig(np.matmul(Ta, np.conjugate(Tb.T)))

In [8]:
eigvals_b, eigvecs_b = np.linalg.eig(np.conjugate(np.transpose(np.matmul(np.conjugate(Tb.T), Ta))))

In [9]:
eigvals_a

array([-2.77555756e-17,  2.32816056e-01,  4.77514952e-01])

In [10]:
eigvals_b

array([0.47751495, 0.23281606])

In [11]:
# pair up eigenvalues of Ta and Tb
idx_pairs = []
tol = 1e-5

for i, eigval_a in enumerate(eigvals_a):
    for j, eigval_b in enumerate(eigvals_b):
        if abs(eigval_a - eigval_b) < tol:
            idx_pairs += [(i, j)]
            break

idx_pairs

[(1, 1), (2, 0)]

In [48]:
nbcomp = len(idx_pairs)
a = np.zeros((nbcomp*x.shape[2], nbcomp*x.shape[2]))
b = np.zeros(nbcomp*x.shape[2])
eqidx = 0
for k in range(x.shape[2]):
    for i, j in random.choices(list(product(range(x.shape[0]), range(x.shape[1]))), k=nbcomp):
        #print("{} {} {}".format(i, j, k))
        b[eqidx] = x[i, j, k]
        for ck in range(nbcomp):
            a[eqidx, ck*nbcomp+k] = eigvecs_a[idx_pairs[ck][0], i]*eigvecs_b[idx_pairs[ck][1], j]
        eqidx += 1

0 0 0
2 0 0
0 0 1
0 1 1


In [51]:
np.linalg.solve(a, b)

array([-7.35669573e+02,  1.54934845e+00,  1.33870024e+00, -6.63460273e-03])